### The PRIMAP-hist national historical emissions time series (1750-2019) v2.3.1

<font size="3">https://zenodo.org/record/5494497#.YujsrTfP2Un</font>

Load Environment Variables

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [2]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_primap_hist_emissions'

In [3]:
# Bucket must be configured with credentials for the Hive ingestion bucket
hive_bucket = osc.attach_s3_bucket('S3_OSCCL2')
hive_catalog = 'osc_datacommons_hive_ingest'
hive_schema = 'ingest'

In [4]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_OSCCL2")

In [5]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [6]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
#print(qres.fetchall())

In [7]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('dera',)
('essd',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('sandbox',)
('wri_gppd',)


In [8]:
import pandas as pd
import csv
import ParseXLS as parser


primap_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/PRIMAP/Guetschow-et-al-2022-PRIMAP-hist_v2.4_11-Oct-2022.csv')
primap_file.download_file(f'/tmp/Guetschow-et-al-2022-PRIMAP-hist_v2.4_11-Oct-2022.csv')
#df = parser.process('OECD_IMGR_FCO2.ini','OECD.csv') 

df = parser.process('PRIMAP.ini','PRIMAP.csv') 
df=df.astype({'validity_date': 'int32'})
df=df.astype({'country_iso_code': 'string'})
df=df.astype({'attribute': 'string'})
df= df.convert_dtypes()
df.info(verbose=True)

#df['validity_date_dt'] = pd.to_datetime(df.validity_date, format='%Y')
#df = df.convert_dtypes()
#df.validity_date_dt = df.validity_date_dt.astype('datetime64[ns]')



PRIMAP.ini
file_list:
['/tmp/Guetschow-et-al-2022-PRIMAP-hist_v2.4_11-Oct-2022.csv']
/tmp/Guetschow-et-al-2022-PRIMAP-hist_v2.4_11-Oct-2022.csv
2
csv
/tmp/Guetschow-et-al-2022-PRIMAP-hist_v2.4_11-Oct-202
                 source scenario (PRIMAP-hist) area (ISO3) entity  \
0      PRIMAP-hist_v2.4                 HISTCR         ABW    CH4   
1      PRIMAP-hist_v2.4                 HISTCR         ABW    CH4   
2      PRIMAP-hist_v2.4                 HISTCR         ABW    CH4   
3      PRIMAP-hist_v2.4                 HISTCR         ABW    CH4   
4      PRIMAP-hist_v2.4                 HISTCR         ABW    CH4   
...                 ...                    ...         ...    ...   
29575  PRIMAP-hist_v2.4                 HISTTP         ZWE    N2O   
29576  PRIMAP-hist_v2.4                 HISTTP         ZWE    N2O   
29577  PRIMAP-hist_v2.4                 HISTTP         ZWE    N2O   
29578  PRIMAP-hist_v2.4                 HISTTP         ZWE    N2O   
29579  PRIMAP-hist_v2.4              

In [9]:
columnschema = osc.create_table_schema_pairs(df,typemap={'datetime64[ns]':'timestamp(6)'}) 
print(columnschema)

'''
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = ARRAY['country_iso_code']
)
"""
print(tabledef)
'''

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
#print(qres.fetchall())

#table_create = engine.execute(tabledef)
#print(table_create.fetchall())




    rec_source varchar,
    data_provider varchar,
    country_name bigint,
    country_iso_code varchar,
    validity_date integer,
    attribute varchar,
    value double,
    value_units varchar

drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_primap_hist_emissions



In [10]:
df.info(verbose=True)
        
        

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58208 entries, 0 to 58207
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rec_source        58208 non-null  string 
 1   data_provider     58208 non-null  string 
 2   country_name      0 non-null      Int64  
 3   country_iso_code  58208 non-null  string 
 4   validity_date     58208 non-null  Int32  
 5   attribute         58208 non-null  string 
 6   value             57971 non-null  Float64
 7   value_units       58208 non-null  string 
dtypes: Float64(1), Int32(1), Int64(1), string(5)
memory usage: 3.9 MB


In [11]:
osc.fast_pandas_ingest_via_hive(
    df,
    engine,
    ingest_catalog, ingest_schema, ingest_table,
    hive_bucket, hive_catalog, hive_schema,
    partition_columns = ['country_iso_code'],
    overwrite = True,
    verbose = True
)

enforcing dataframe partition column order

verifying existence of table osc_datacommons_dev.pcaf_sovereign_footprint.sf_primap_hist_emissions
create table if not exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_primap_hist_emissions (
    rec_source varchar,
    data_provider varchar,
    country_name bigint,
    validity_date integer,
    attribute varchar,
    value double,
    value_units varchar,
    country_iso_code varchar
) with (
    format = 'parquet',
    partitioning = array['country_iso_code']
)

staging dataframe parquet to s3 osc-datacommons-s3-bucket-dev02
/tmp/ingest_temp_5ea374fd/country_iso_code=ABW/8d63125d156547c8ac791f0c41f5b14f-0.parquet  -->  trino/ingest/ingest_temp_5ea374fd/country_iso_code=ABW/8d63125d156547c8ac791f0c41f5b14f-0.parquet
/tmp/ingest_temp_5ea374fd/country_iso_code=AFG/8d63125d156547c8ac791f0c41f5b14f-0.parquet  -->  trino/ingest/ingest_temp_5ea374fd/country_iso_code=AFG/8d63125d156547c8ac791f0c41f5b14f-0.parquet
/tmp/ingest_temp_5ea374fd/c

In [12]:
'''
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1, verbose = True))
'''           

"\ndf.to_sql(ingest_table,\n           con=engine,\n           schema=ingest_schema,\n           if_exists='append',\n           index=False,\n           method=osc.TrinoBatchInsert(batch_size = 1, verbose = True))\n"

In [15]:
import pandas as pd
sql=f"""
select *  from {ingest_catalog}.{ingest_schema}.{ingest_table} where validity_date=2021 and country_iso_code='ARG'"""
pd.read_sql(sql, engine)

,rec_source,data_provider,country_name,validity_date,attribute,value,value_units,country_iso_code
0,Guetschow-et-al-2022-PRIMAP-hist_v2.4_11-Oct-2...,PRIMAP,None,2021,CO2,196000.0,CO2 * gigagram / a,ARG
